In [ ]:
#!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
"""
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric
"""

'\n!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-geometric\n'

In [ ]:
from torch.autograd import Variable
import torch_geometric.transforms
from torch_geometric.transforms import knn_graph
import torch_geometric.data
import torch 
from torch_geometric.data import Data
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
import torch
import pyarrow as pa
import pyarrow.parquet as pq
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from torch_geometric.nn import knn_graph
import os 

from torch.utils.data import *
from functools import partial
import timeit

In [ ]:
import argparse
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=50, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=25, help='Initial learning rate.') #100

parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.') #0.001
parser.add_argument('--dropout', type=float, default=0.4, help='Dropout rate (1 - keep probability).')
args = parser.parse_args([])
torch.backends.cudnn.benchmark = True

In [ ]:
class ParquetDataset(Dataset):
    def __init__(self, filename):
        self.parquet = pq.ParquetFile(filename)
        self.cols = None 
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        data['X_jets'] = np.float32(data['X_jets'][0]) 
        # Preprocessing
        data['X_jets'] = data['X_jets'][1]
        data['X_jets'][data['X_jets'] < 1.e-3] = 0. # Zero-Suppression
        return dict(data)
    def __len__(self):
        return self.parquet.num_row_groups

In [ ]:
def jets(i, ecal):
    xhit2,yhit2 = torch.nonzero(ecal[i])[:, 0].cpu().numpy(), torch.nonzero(ecal[i])[:, 1].cpu().numpy()    
    eneEcal=ecal[i].cpu().numpy()[xhit2,yhit2]
    feats=np.transpose(np.vstack((xhit2,yhit2,eneEcal)))  ## concatenate x,y locations and energies (3 features in total)
    cords=feats[:,[0,1]] ## cords = x,y coordinates 
    allFeats=torch.from_numpy(feats).float()  ## features to tensors 
    cords2=torch.from_numpy(cords)  ## coordinates to tensors 
    edge_index = knn_graph(cords2, k=4, batch=None, loop=True)  ## Create knn graph adjacency matrix 
    data=Data(x=allFeats,edge_index=edge_index) ## Create graph data with feature matrix x and adjacency matrix edge_index
    return data

In [ ]:
def get_data_loader(datasets, batch_size, cut, random_sampler=False):
    dset = ConcatDataset([ParquetDataset(dataset) for dataset in datasets])
    idxs = np.random.permutation(len(dset))
    if random_sampler: 
        random_sampler = sampler.SubsetRandomSampler(idxs[:cut])
    else: 
        random_sampler = None 
    data_loader = DataLoader(dataset=dset, batch_size=batch_size, shuffle=False, num_workers=0, sampler=random_sampler, pin_memory=True)
    return data_loader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/My\ Drive/parquets

/content/drive/My Drive/parquets


In [ ]:
datasets = ['Boosted_Jets_Sample-%i.snappy.parquet'%i for i in range(5)]
data_loader = get_data_loader(datasets, args.batch_size, cut = None, random_sampler = True)

In [ ]:
def jets(datei,number1,number2):

    cols = None
    graphs=[]

    for i in range(number1,number2):
      #datei['X_jets'] = np.float32(datei['X_jets']) [0] 
      
      #ecal=datei['X_jets'][1]  ## Select ECAL Data out of the 3 available channels

      #ecal[ecal<=1e-3]=0  ## Remove noisy values
      ecal=datei[i].cpu().numpy()

      xhit2,yhit2=np.nonzero(ecal)  ## Select hits in detector

      eneEcal=ecal[xhit2,yhit2]  ## Select energies of hits

      feats=np.transpose(np.vstack((xhit2,yhit2,eneEcal)))  ## concatenate x,y locations and energies (3 features in total)
      
      cords=feats[:,[0,1]] ## cords = x,y coordinates 
      
      allFeats=torch.from_numpy(feats).float()  ## features to tensors 
      
      cords2=torch.from_numpy(cords)  ## coordinates to tensors 
    
      edge_index = knn_graph(cords2, k=4, batch=None, loop=True)  ## Create knn graph adjacency matrix 

      donnees=Data(x=allFeats,edge_index=edge_index) ## Create graph data with feature matrix x and adjacency matrix edge_index
      
      graphs.append(donnees)

    return graphs


In [ ]:
cd ../v22/Version2_Pooling

/content/drive/My Drive/v22/Version2_Pooling


In [ ]:
import DiffAE

In [ ]:
model3=DiffAE.GraphAE()
model3.train()
model3.to(device)#.cuda()

GraphAE(
  (gnn1_pool): GNN(
    (conv1): DenseSAGEConv(3, 64)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): DenseSAGEConv(64, 612)
    (bn3): BatchNorm1d(612, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (lin): Linear(in_features=676, out_features=612, bias=True)
  )
  (gnn1_embed): GNN(
    (conv1): DenseSAGEConv(3, 64)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): DenseSAGEConv(64, 64)
    (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (gnn2_embed): GNN(
    (conv1): DenseSAGEConv(128, 64)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): DenseSAGEConv(64, 64)
    (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (gnn3_embed): GNN(
    (conv1): DenseSAGEConv(128, 64)
    (bn1): BatchNorm1d(64, eps=1e-05, m

In [ ]:
optimizer = torch.optim.Adam(model3.parameters(), lr=args.lr, weight_decay=0.001)

In [ ]:

## generate list to count nodes for each graph
def nodeCounter(samples):
    inds=[]
    for k in samples:
        inds.append(k['x'].shape[0])
    return inds

In [ ]:
def ref(bsize,nodeC,i1,i2):
  maxC=np.max(np.array(nodeC))
  maxC=maxC + (4 - maxC % 4) ##max num of nodes 1161%4
  refMat=np.zeros((bsize,maxC)) ## matrix of zeros
  for pi in range(i1,i2):##10
    refMat[10-(i2-pi),:nodeC[pi]]=1 ## fill ones 
  return refMat,maxC

In [ ]:
def assigner(nodelist):
  fin=[]
  countit=0
  for m in nodelist:
      fin.append(np.repeat(countit,m))
      countit+=1
  return np.array(fin)

In [ ]:
from optimizer import loss_function

In [ ]:
import time
for epoch in range(50):
      #model.train()
  count=0
  c1,c2=0,args.batch_size
  epLoss=0
  t = time.time()
  for i, data in enumerate(data_loader):
        ecal2 = data['X_jets'].cuda()
        rawGraph=jets(ecal2,0,25) ##Generating graphs from raw data 
        nodeCount=nodeCounter(rawGraph)
        lengs=torch.LongTensor(np.hstack(assigner(np.array(nodeCount[c1:c2])-c1))).cuda()
        
        compress=torch_geometric.data.Batch.from_data_list(rawGraph)

        gra=compress.x.to(device)
        adj=compress.edge_index.to(device)

        count+=1
        refMat,maxCount=ref(args.batch_size,nodeCount,c1,c2)
        
        optimizer.zero_grad()
        mask=torch.from_numpy(refMat).to(device)
        maxi=torch.from_numpy(np.array(maxCount)).to(device)

        r1 ,adj1,mu,sig= model3(gra,adj,lengs.to(device),mask,maxi)

        #sparse=to_sparse_batch(r1, adj1, mask=torch.LongTensor(mask).cuda())

        loss = loss_function(r1,gra,lengs,refMat,mu,sig)    

        loss.backward()

        optimizer.step()

        cur_loss = loss.item()
        
        epLoss+=float(cur_loss)

        #c1+=args.batch_size
        #c2+=args.batch_size
        count+=1
        if count%1160==0:
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(epLoss/count),"time=", "{:.5f}".format(time.time() - t))
            t = time.time()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 0001 train_loss= 1041.17223 time= 437.43640
Epoch: 0001 train_loss= 913.56024 time= 433.07102
Epoch: 0001 train_loss= 873.35271 time= 431.05122
Epoch: 0001 train_loss= 835.85059 time= 429.85847
Epoch: 0001 train_loss= 815.05890 time= 434.14579
Epoch: 0001 train_loss= 792.83386 time= 426.00362
Epoch: 0001 train_loss= 770.70140 time= 427.75365
Epoch: 0001 train_loss= 750.00343 time= 426.23924


In [ ]:
count

62

In [ ]:
np.where(lengs.cpu().numpy()==0)[0].shape[0]

1170

In [ ]:
lengs

tensor([ 0,  0,  0,  ..., 24, 24, 24], device='cuda:0')

In [ ]:
581/2

In [ ]:
291

In [ ]:
291*4

In [ ]:
model3.is_cuda

In [ ]:

for epoch in range(50):
    #model.train()
    
    count=0
    c1,c2=0,args.batch_size
    epLoss=0
    t = time.time()
    for el in islice(loader,0,2320):#count2,el in enumerate(loader):
        
        gra=el.x
        adj=el.edge_index

        lengs=torch.LongTensor(np.hstack(np.array(fin[c1:c2])-c1))
        count+=1
        refMat=ref(args.batch_size,maxCount,c1,c2)
        #whole=to_dense_batch(el.x, lengs, fill_value=0, max_num_nodes=refMat.shape[1])
        #wholeAdj=to_dense_adj(adj, lengs, edge_attr=None, max_num_nodes=refMat.shape[1])

        optimizer.zero_grad()
        mask=torch.from_numpy(refMat).cuda()
        r1 ,adj1,mu,sig= model3(gra.cuda(),adj.cuda(),lengs.cuda(),mask)

        #sparse=to_sparse_batch(r1, adj1, mask=torch.LongTensor(mask).cuda())

        loss = loss_function(r1,gra,lengs,refMat,mu,sig)    

        loss.backward()

        optimizer.step()

        cur_loss = loss.item()
        
        epLoss+=float(cur_loss)

        c1+=args.batch_size
        c2+=args.batch_size
        
        if count%1160==0:
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(epLoss/count),"time=", "{:.5f}".format(time.time() - t))
            t = time.time()

            with torch.no_grad():
              model3.eval()
              VaLoss=0
              d1,d2=0,args.batch_size
              for el2 in islice(Testloader,40):
                gra2=el2.x.cuda()  ##features 
                adj2=el2.edge_index.cuda() ## edge matrix 
                lengs2=torch.LongTensor(np.hstack(np.array(fin2[d1:d2])-d1)).cuda()
                refMat2=ref(args.batch_size,maxCount,d1,d2)

                r12,mu2,sig2= model3(gra2,adj2,lengs2,torch.from_numpy(refMat2).cuda()) ## X , A 
                loss2 = loss_function(r12,gra2,lengs2,refMat,mu2,sig2)
                cur_loss2 = loss2.item()
                VaLoss+=float(cur_loss2)
                d1+=args.batch_size
                d2+=args.batch_size
              print("Val Loss:", '%04d' % (VaLoss/10))

    if epoch%2==0:
      scheduler.step()
      torch.save({
            'epoch': epoch,
            'model_state_dict': model3.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'epoch':epoch,
            'loss': loss,
            'epLoss':epLoss
            }, './diffPool.pth')

           